# EE502 Project
## Use DREAMER dataset to create a ML model that will classify ECG data in a range of emotions responses
## Ernesto Sola-Thomas 
## Fall 2023 - Clarkson University

In [24]:
# Nvidia RAPIDS (GPU accelerated training)
import cuml
import cudf

from cuml.model_selection import train_test_split
from cuml.ensemble import RandomForestClassifier
from cuml.svm import SVC
from cuml.svm import LinearSVC
from cuml.linear_model import LogisticRegression
from cuml.decomposition import PCA


# ECG Raw data processing tool
import heartpy as hp

# Pickle to read and write files with dataset 
import pickle

# Standard Libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import itertools
import copy

In [2]:
import warnings

# Filter out specific RuntimeWarning from numpy
warnings.filterwarnings('ignore', category=RuntimeWarning, module='numpy')

# Filter out specific UserWarning from scipy
warnings.filterwarnings('ignore', category=UserWarning, module='scipy')

# Define Functions

In [19]:
# Global Parameters
num_of_participants = 23
num_of_clips = 18

#In Hz:
cutoff = [0.5, 45]
sample_rate = 256.0
order = 3
filtertype ='bandpass'

# Features
window_size = 20480

In [20]:
# Global Functions

"""
data_filter takes in the RAW ECG data and will filter out noice and remove DC offset, see parameters defined above to finetune filter.

Input:  pandas dataframe with RAW ECG data 
Output: pandas dataframe with filtered/denoised data
"""
def data_filter (data_in):
    data = dict(data_in)
    # Separate data by channel
    for participant in data.keys():
        #print("Starting " + participant)
        for clip_num in range(0, num_of_clips):
            # Filter both channels from baseline
            CH1 = [row[0] for row in data[participant]['ECG']['baseline'][clip_num]]
            CH1_filtered = hp.filter_signal(CH1, cutoff = cutoff, sample_rate = sample_rate, order = order, filtertype=filtertype)
            # CH1_filtered = hp.butter_lowpass_filter(data, cutoff=cutoff, sample_rate=sample_rate, order=order)

            CH2 = [row[1] for row in data[participant]['ECG']['baseline'][clip_num]]
            CH2_filtered = hp.filter_signal(CH2, cutoff = cutoff, sample_rate = sample_rate, order = order, filtertype=filtertype)
            # CH2_filtered = hp.butter_lowpass_filter(data, cutoff=cutoff, sample_rate=sample_rate, order=order)

            s_clip = "clip " + str(clip_num+1) 

            data[participant]['ECG'][s_clip] = {}
            data[participant]['ECG'][s_clip]['CH1'] = {}
            data[participant]['ECG'][s_clip]['CH1']['baseline'] = CH1_filtered.tolist()
            data[participant]['ECG'][s_clip]['CH2'] = {}
            data[participant]['ECG'][s_clip]['CH2']['baseline'] = CH2_filtered.tolist()

            
            # Filter both channels from stimuli
            CH1 = [row[0] for row in data[participant]['ECG']['stimuli'][clip_num]]
            CH1_filtered = hp.filter_signal(CH1, cutoff = cutoff, sample_rate = sample_rate, order = order, filtertype=filtertype)
            # CH1_filtered = hp.butter_lowpass_filter(data, cutoff=cutoff, sample_rate=sample_rate, order=order)

            CH2 = [row[1] for row in data[participant]['ECG']['stimuli'][clip_num]]
            CH2_filtered = hp.filter_signal(CH2, cutoff = cutoff, sample_rate = sample_rate, order = order, filtertype=filtertype)
            # CH2_filtered = hp.butter_lowpass_filter(data, cutoff=cutoff, sample_rate=sample_rate, order=order)

            data[participant]['ECG'][s_clip]['CH1']['stimuli'] = CH1_filtered.tolist()
            data[participant]['ECG'][s_clip]['CH2']['stimuli'] = CH2_filtered.tolist()

            data[participant]['ECG'][s_clip]['ScoreValence'] = data[participant]['ScoreValence'][clip_num]
            data[participant]['ECG'][s_clip]['ScoreArousal'] = data[participant]['ScoreArousal'][clip_num]
            data[participant]['ECG'][s_clip]['ScoreDominance'] = data[participant]['ScoreDominance'][clip_num]
            
            print("Done filtering raw for " + participant + " at clip#" + str(clip_num+1) + "                 ", end="\r", flush=True)
        data[participant]['ECG'].pop('stimuli')
        data[participant]['ECG'].pop('baseline')

        data[participant].pop('ScoreValence')
        data[participant].pop('ScoreArousal')
        data[participant].pop('ScoreDominance')
    
    print("All done.... Highpass filter applied, cutoff: [" + str(cutoff[0]) + "," + str(cutoff[1]) + "]Hz")
    return data

"""
extract_ecg_features takes in the denoised ECG data and will extract features to be used to train the ML model.
The extracted features include: 
"""
def extract_ecg_features(processed_data):
    combined_data = pd.DataFrame()
    sample_rate = 256  # Assuming 256 is the sample rate

    for CH in ['CH1', 'CH2']:
        for participant in processed_data:
            for clip in processed_data[participant]['ECG']:
                print("Processing " + participant + " " + clip + " " + CH + "          ", end="\r", flush=True)
                stimuli_data = processed_data[participant]['ECG'][clip][CH]['stimuli']

                # Iterate over the data in windows of size window_size
                for start in range(0, len(stimuli_data), window_size):
                    end = start + window_size
                    window_data = stimuli_data[start:end]

                    # Check if the window has enough data, if not, skip this window
                    if len(window_data) < window_size:
                        continue

                    # Process the windowed data
                    hp_tupple = hp.process(window_data, sample_rate)
                    features_df = pd.DataFrame([hp_tupple[1]])
                    features_df['ScoreValence'] = processed_data[participant]['ECG'][clip]['ScoreValence']
                    features_df['ScoreArousal'] = processed_data[participant]['ECG'][clip]['ScoreArousal']
                    features_df['ScoreDominance'] = processed_data[participant]['ECG'][clip]['ScoreDominance']

                    # Append the features to the combined data
                    combined_data = pd.concat([combined_data, features_df], ignore_index=True)

    print("\nDone extracting time domain features")
    return combined_data

# Load Data
### This scrip assumes all the raw ECG data has been pre-extracted and merged in a single directory and saved in a .pkl file

In [ ]:
!python create_raw_json_and_pkl.py

In [5]:
raw_data = {}
load_raw_from = '../../data_set_raw_ECG/raw_data_dict.pkl'
# To generate ^^ Run create_raw_json_and_pkl.py

with open(load_raw_from, 'rb') as f:
    raw_data = pickle.load(f)

filtered_data = copy.deepcopy(raw_data)
filtered_data = data_filter(filtered_data)

fig, axs = plt.subplots(1, 2)
axs[0].plot(raw_data["participant_1"]['ECG']['stimuli'][0][0:1024], label=["CH1", "CH2"])
axs[0].set_title("RAW DATA")

axs[1].plot(filtered_data['participant_1']['ECG']['clip 1']['CH1']['stimuli'][0:1024])
axs[1].plot(filtered_data['participant_1']['ECG']['clip 1']['CH2']['stimuli'][0:1024])
axs[1].set_title("Filtered Data")

All done.... Highpass filter applied, cutoff: [0.5,45]Hz         


Text(0.5, 1.0, 'Filtered Data')

# Extract ECG Features

In [21]:
ECG_features = extract_ecg_features(filtered_data)
ECG_features

Processing participant_9 clip 18 CH2           
Done extracting time domain features


bpm         ibi       sdnn       sdsd      rmssd     pnn20  \
0     80.509454  745.254089  37.015695  14.440489  23.468189  0.367925   
1     80.705859  743.440448  48.846686  19.889675  29.236669  0.371429   
2     76.372766  785.620359  65.265267  21.760162  34.277205  0.470000   
3     76.489498  784.421411  37.990842  16.527454  27.792083  0.400000   
4     80.550141  744.877653  60.682510  15.128795  23.517967  0.371429   
...         ...         ...        ...        ...        ...       ...   
1743  72.013029  833.182566  21.107251  10.879065  17.964052  0.255319   
1744  72.339089  829.427083  26.195833  10.703118  16.865822  0.200000   
1745  76.360925  785.742188  29.560661   8.893522  15.540928  0.181818   
1746  67.241071  892.311790  27.285319  14.795161  24.825688  0.425287   
1747  70.085369  856.098790  29.512620  12.509273  24.203405  0.445652   

         pnn50     hr_mad        sd1        sd2            s   sd1/sd2  \
0     0.037736  23.437500  16.589278  49.718637  2591.173886  0.333663   
1     0.076190  33.203125  20.673447  66.233059  4301.674892  0.312132   
2     0.160000  46.875000  24.219443  88.898887  6764.105105  0.272438   
3     0.100000  27.343750  19.632083  49.403825  3047.030759  0.397380   
4     0.038095  39.062500  16.628382  84.576428  4418.239095  0.196608   
...        ...        ...        ...        ...          ...       ...   
1743  0.000000  11.718750  12.697608  26.729556  1066.260979  0.475040   
1744  0.010526  15.625000  11.925902  35.170987  1317.727633  0.339084   
1745  0.000000  19.531250  10.988776  40.471877  1397.180651  0.271516   
1746  0.057471  17.578125  17.548784  34.209326  1885.998807  0.512982   
1747  0.010870  19.531250  17.113444  38.070385  2046.795964  0.449521   

      breathingrate  ScoreValence  ScoreArousal  ScoreDominance  
0          0.100000             4             3               2  
1          0.133333             4             3               2  
2          0.133333             3             3               1  
3          0.266667             5             4               4  
4          0.266667             5             4               4  
...             ...           ...           ...             ...  
1743       0.300000             2             4               4  
1744       0.300000             2             4               4  
1745       0.300000             2             4               4  
1746       0.266667             3             4               4  
1747       0.300000             3             4               4  

[1748 rows x 16 columns]

# Define Models (WITH NO PCA)

In [26]:
# Define your feature set and labels
cudf_ECG_features = cudf.DataFrame.from_pandas(ECG_features)
X = cudf_ECG_features[['bpm', 'ibi', 'sdnn', 'sdsd', 'rmssd', 'pnn20', 'pnn50', 'hr_mad', 'sd1', 'sd2', 's', 'sd1/sd2', 'breathingrate']]
Y_valence = cudf_ECG_features['ScoreValence']
Y_arousal = cudf_ECG_features['ScoreArousal']
Y_dominance = cudf_ECG_features['ScoreDominance']

# Initialize variables to track the best model and parameters for each label
best_models = {
    'ScoreValence': {'score': float('-inf'), 'model': None, 'params': None},
    'ScoreArousal': {'score': float('-inf'), 'model': None, 'params': None},
    'ScoreDominance': {'score': float('-inf'), 'model': None, 'params': None}
}

# Define a range of hyperparameters for each algorithm
hyperparameters = {
    RandomForestClassifier: {'n_estimators': [100, 200, 300, 400], 'max_depth': [5, 10, 15, 20, 25]},
    SVC: {'C': [1, 10, 100], 'kernel': ['rbf']},
    LinearSVC: {'C': [1, 10, 100]},
    LogisticRegression: {'C': [1, 10, 100]}
    # Add other algorithms and their hyperparameters here
}

# Full search
for algorithm in hyperparameters:
    print("~~~~~~~~~~~~~~~~~~~algorithm " + str(algorithm) + "~~~~~~~~~~~~~~~~~~~")
    for param_combination in itertools.product(*hyperparameters[algorithm].values()):
        params = dict(zip(hyperparameters[algorithm].keys(), param_combination))

        # Train a model for each label
        for output in ['ScoreValence', 'ScoreArousal', 'ScoreDominance']:
            if output == 'ScoreValence':
                Y = Y_valence
            elif output == 'ScoreArousal':
                Y = Y_arousal
            elif output == 'ScoreDominance':
                Y = Y_dominance

            # Split data
            X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

            # Initialize and train model
            model = algorithm(**params)

            # Convert DataFrame/Series to arrays if necessary
            X_train_array = X_train.values
            Y_train_array = Y_train.values

            # Fit the model
            model.fit(X_train_array, Y_train_array)

            # Evaluate model
            score = model.score(X_test, Y_test)

            # Update best_models if current model is better
            if score > best_models[output]['score']:
                best_models[output]['score'] = score
                best_models[output]['model'] = algorithm.__name__
                best_models[output]['params'] = params

            print(f"{output}, Algorithm {algorithm.__name__}, Params: {params}, Label: {Y.name}, Score: {score}")

# Analyze results to determine the best model for each label
# Print the best models and their hyperparameters for each label
print("\n\n~~~~~~~~~~~~~~~~~~~FINISED TRANING MODELS~~~~~~~~~~~~~~~~~~~\n\n")

for label, info in best_models.items():
    print(f"Best Model for {label}: {info['model']} with Score: {info['score']}")
    print(f"Hyperparameters: {info['params']}\n")

~~~~~~~~~~~~~~~~~~~algorithm <class 'cuml.ensemble.randomforestclassifier.RandomForestClassifier'>~~~~~~~~~~~~~~~~~~~


/opt/conda/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


ScoreValence, Algorithm RandomForestClassifier, Params: {'n_estimators': 100, 'max_depth': 5}, Label: ScoreValence, Score: 0.3381088972091675
ScoreArousal, Algorithm RandomForestClassifier, Params: {'n_estimators': 100, 'max_depth': 5}, Label: ScoreArousal, Score: 0.4813753664493561
ScoreDominance, Algorithm RandomForestClassifier, Params: {'n_estimators': 100, 'max_depth': 5}, Label: ScoreDominance, Score: 0.40401145815849304
ScoreValence, Algorithm RandomForestClassifier, Params: {'n_estimators': 100, 'max_depth': 10}, Label: ScoreValence, Score: 0.6131805181503296
ScoreArousal, Algorithm RandomForestClassifier, Params: {'n_estimators': 100, 'max_depth': 10}, Label: ScoreArousal, Score: 0.68767911195755
ScoreDominance, Algorithm RandomForestClassifier, Params: {'n_estimators': 100, 'max_depth': 10}, Label: ScoreDominance, Score: 0.6131805181503296
ScoreValence, Algorithm RandomForestClassifier, Params: {'n_estimators': 100, 'max_depth': 15}, Label: ScoreValence, Score: 0.782234966754

# Define Models (WITH PCA)

In [30]:
# Initialize variables to track the best model and parameters for each label
best_models_PCA = {
    'ScoreValence': {'score': float('-inf'), 'model': None, 'params': None, 'PCA': None},
    'ScoreArousal': {'score': float('-inf'), 'model': None, 'params': None, 'PCA': None},
    'ScoreDominance': {'score': float('-inf'), 'model': None, 'params': None, 'PCA': None}
}

# Rest of your script for model selection and evaluation
for PCA_components in [2,3,4,5,6,7,8,9,10]:
    # Apply PCA transformation
    pca = PCA(n_components=PCA_components)
    X_pca = pca.fit_transform(X)

    for algorithm in hyperparameters:
        print("~~~~~~~~~~~~~~~~~~~algorithm " + str(algorithm) + "~~~~~~~~~~~~~~~~~~~")
        for param_combination in itertools.product(*hyperparameters[algorithm].values()):
            params = dict(zip(hyperparameters[algorithm].keys(), param_combination))

            # Train a model for each label
            for output in ['ScoreValence', 'ScoreArousal', 'ScoreDominance']:
                if output == 'ScoreValence':
                    Y = Y_valence
                elif output == 'ScoreArousal':
                    Y = Y_arousal
                elif output == 'ScoreDominance':
                    Y = Y_dominance

                # Split data
                X_train, X_test, Y_train, Y_test = train_test_split(X_pca, Y, test_size=0.2)

                # Initialize and train model
                model = algorithm(**params)

                # Convert DataFrame/Series to arrays if necessary
                X_train_array = X_train.values
                Y_train_array = Y_train.values

                # Fit the model
                model.fit(X_train_array, Y_train_array)

                # Evaluate model
                score = model.score(X_test, Y_test)

                # Update best_models if current model is better
                if score > best_models_PCA[output]['score']:
                    best_models_PCA[output]['score'] = score
                    best_models_PCA[output]['model'] = algorithm.__name__
                    best_models_PCA[output]['params'] = params
                    best_models_PCA[output]['PCA'] = PCA_components

                print(f"{output}, Algorithm {algorithm.__name__}, PCA: {PCA_components}, Params: {params}, Label: {Y.name}, Score: {score}")

# Analyze results to determine the best model for each label
# Print the best models and their hyperparameters for each label
print("\n\n~~~~~~~~~~~~~~~~~~~FINISED TRANING MODELS~~~~~~~~~~~~~~~~~~~\n\n")

for label, info in best_models_PCA.items():
    print(f"Best Model for {label}: {info['model']} with Score: {info['score']}")
    print(f"Hyperparameters: {info['params']}\n")

~~~~~~~~~~~~~~~~~~~algorithm <class 'cuml.ensemble.randomforestclassifier.RandomForestClassifier'>~~~~~~~~~~~~~~~~~~~


/opt/conda/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


ScoreValence, Algorithm RandomForestClassifier, PCA: 2, Params: {'n_estimators': 100, 'max_depth': 5}, Label: ScoreValence, Score: 0.2435530126094818
ScoreArousal, Algorithm RandomForestClassifier, PCA: 2, Params: {'n_estimators': 100, 'max_depth': 5}, Label: ScoreArousal, Score: 0.36103153228759766
ScoreDominance, Algorithm RandomForestClassifier, PCA: 2, Params: {'n_estimators': 100, 'max_depth': 5}, Label: ScoreDominance, Score: 0.3553008735179901
ScoreValence, Algorithm RandomForestClassifier, PCA: 2, Params: {'n_estimators': 100, 'max_depth': 10}, Label: ScoreValence, Score: 0.30085960030555725
ScoreArousal, Algorithm RandomForestClassifier, PCA: 2, Params: {'n_estimators': 100, 'max_depth': 10}, Label: ScoreArousal, Score: 0.32951289415359497
ScoreDominance, Algorithm RandomForestClassifier, PCA: 2, Params: {'n_estimators': 100, 'max_depth': 10}, Label: ScoreDominance, Score: 0.4097421169281006
ScoreValence, Algorithm RandomForestClassifier, PCA: 2, Params: {'n_estimators': 100, 

/opt/conda/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


ScoreValence, Algorithm RandomForestClassifier, PCA: 3, Params: {'n_estimators': 100, 'max_depth': 5}, Label: ScoreValence, Score: 0.3467048704624176
ScoreArousal, Algorithm RandomForestClassifier, PCA: 3, Params: {'n_estimators': 100, 'max_depth': 5}, Label: ScoreArousal, Score: 0.3868194818496704
ScoreDominance, Algorithm RandomForestClassifier, PCA: 3, Params: {'n_estimators': 100, 'max_depth': 5}, Label: ScoreDominance, Score: 0.40401145815849304
ScoreValence, Algorithm RandomForestClassifier, PCA: 3, Params: {'n_estimators': 100, 'max_depth': 10}, Label: ScoreValence, Score: 0.4297994375228882
ScoreArousal, Algorithm RandomForestClassifier, PCA: 3, Params: {'n_estimators': 100, 'max_depth': 10}, Label: ScoreArousal, Score: 0.467048704624176
ScoreDominance, Algorithm RandomForestClassifier, PCA: 3, Params: {'n_estimators': 100, 'max_depth': 10}, Label: ScoreDominance, Score: 0.467048704624176
ScoreValence, Algorithm RandomForestClassifier, PCA: 3, Params: {'n_estimators': 100, 'max

...

ScoreValence, Algorithm RandomForestClassifier, PCA: 8, Params: {'n_estimators': 100, 'max_depth': 5}, Label: ScoreValence, Score: 0.38108882308006287
ScoreArousal, Algorithm RandomForestClassifier, PCA: 8, Params: {'n_estimators': 100, 'max_depth': 5}, Label: ScoreArousal, Score: 0.42406877875328064
ScoreDominance, Algorithm RandomForestClassifier, PCA: 8, Params: {'n_estimators': 100, 'max_depth': 5}, Label: ScoreDominance, Score: 0.3896848261356354
ScoreValence, Algorithm RandomForestClassifier, PCA: 8, Params: {'n_estimators': 100, 'max_depth': 10}, Label: ScoreValence, Score: 0.5530086159706116
ScoreArousal, Algorithm RandomForestClassifier, PCA: 8, Params: {'n_estimators': 100, 'max_depth': 10}, Label: ScoreArousal, Score: 0.6389684677124023
ScoreDominance, Algorithm RandomForestClassifier, PCA: 8, Params: {'n_estimators': 100, 'max_depth': 10}, Label: ScoreDominance, Score: 0.6131805181503296
ScoreValence, Algorithm RandomForestClassifier, PCA: 8, Params: {'n_estimators': 100, '

/opt/conda/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


ScoreValence, Algorithm RandomForestClassifier, PCA: 9, Params: {'n_estimators': 100, 'max_depth': 5}, Label: ScoreValence, Score: 0.343839555978775
ScoreArousal, Algorithm RandomForestClassifier, PCA: 9, Params: {'n_estimators': 100, 'max_depth': 5}, Label: ScoreArousal, Score: 0.41547277569770813
ScoreDominance, Algorithm RandomForestClassifier, PCA: 9, Params: {'n_estimators': 100, 'max_depth': 5}, Label: ScoreDominance, Score: 0.41260746121406555
ScoreValence, Algorithm RandomForestClassifier, PCA: 9, Params: {'n_estimators': 100, 'max_depth': 10}, Label: ScoreValence, Score: 0.6074498295783997
ScoreArousal, Algorithm RandomForestClassifier, PCA: 9, Params: {'n_estimators': 100, 'max_depth': 10}, Label: ScoreArousal, Score: 0.593123197555542
ScoreDominance, Algorithm RandomForestClassifier, PCA: 9, Params: {'n_estimators': 100, 'max_depth': 10}, Label: ScoreDominance, Score: 0.5959885120391846
ScoreValence, Algorithm RandomForestClassifier, PCA: 9, Params: {'n_estimators': 100, 'ma

/opt/conda/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


ScoreValence, Algorithm RandomForestClassifier, PCA: 10, Params: {'n_estimators': 100, 'max_depth': 5}, Label: ScoreValence, Score: 0.37249284982681274
ScoreArousal, Algorithm RandomForestClassifier, PCA: 10, Params: {'n_estimators': 100, 'max_depth': 5}, Label: ScoreArousal, Score: 0.45272207260131836
ScoreDominance, Algorithm RandomForestClassifier, PCA: 10, Params: {'n_estimators': 100, 'max_depth': 5}, Label: ScoreDominance, Score: 0.46418339014053345
ScoreValence, Algorithm RandomForestClassifier, PCA: 10, Params: {'n_estimators': 100, 'max_depth': 10}, Label: ScoreValence, Score: 0.6217765212059021
ScoreArousal, Algorithm RandomForestClassifier, PCA: 10, Params: {'n_estimators': 100, 'max_depth': 10}, Label: ScoreArousal, Score: 0.6074498295783997
ScoreDominance, Algorithm RandomForestClassifier, PCA: 10, Params: {'n_estimators': 100, 'max_depth': 10}, Label: ScoreDominance, Score: 0.6217765212059021
ScoreValence, Algorithm RandomForestClassifier, PCA: 10, Params: {'n_estimators'